__Create 2 cross TE model__

Daniel Köhn
Kiel, 17/08/2017

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter
from scipy.ndimage.filters import gaussian_filter
import matplotlib.image as mpimg
import pickle
#%matplotlib notebook

__Define fonts__

In [ ]:
FSize = 15
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 9, 3

__Read 2 cross TE image__

In [ ]:
fig1 = plt.figure(figsize=(12, 6))
img=mpimg.imread('model/2_cross_TE_model.png')
imgplot = plt.imshow(img)
plt.show()

__FD grid dimensions__

In [ ]:
NY, NX, tmp = np.shape(img)
tmp=None
DH = 0.07

__Define material parameters__

In [ ]:
npara = 3  # number of model parameters

# allocate memory
R = np.ones((npara,1))
G = np.ones((npara,1))
B = np.ones((npara,1))
sigma = np.ones((npara,1))
epsilonr = np.ones((npara,1))

# link colors in the image with material parameters
R = (img[2,2][0], img[64,115][0], img[104,77][0])
G = (img[2,2][1], img[64,115][1], img[104,77][1])
B = (img[2,2][2], img[64,115][2], img[104,77][2])

sigma = (3.5, 10.0, 0.0)
epsilonr = (4.0, 8.0, 1.0)

__Define first version of elastic model__

In [ ]:
sigma0 = 1000.0 * np.ones((NY,NX))
epsilonr0 = np.ones((NY,NX))

# loop over image and replace colors by model parameters
for j in range(0,NY,1):
    
    for i in range(0,NX,1):
        
        for k in range(npara):
            
            if img[j,i][0] == R[k] and img[j,i][1] == G[k] and  img[j,i][2] == B[k]:
            
                sigma0[j,i] = sigma[k]
                epsilonr0[j,i] = epsilonr[k]                

__Define SubPlot__

In [ ]:
def do_plot(n, model, cm, an, title, vpmin, vpmax):
    
    ax=plt.subplot(1, 2, n)
    
    #plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    ## for Palatino and other serif fonts use:
    #rc('font',**{'family':'serif','serif':['Palatino']})
    #plt.rc('text', usetex=True)
    rc('text', usetex=True)
    
    plt.imshow(model, cmap=cm, interpolation='none', extent=[DH,NX*DH,DH,NY*DH], vmin=vpmin, vmax=vpmax)
    a = gca()
    #a.set_xticklabels(a.get_xticks(), font)
    #a.set_yticklabels(a.get_yticks(), font)
    #plt.axis('scaled')
    plt.ylabel('Depth [m]', fontdict=font)
    plt.xlabel('Distance [m]', fontdict=font)
    plt.gca().invert_yaxis()
    cbar=plt.colorbar(aspect=8, pad=0.05)
    cbar.set_label(title, fontdict=font, labelpad=10)
    plt.text(0.8, 1.5,an,fontdict=font,color='white')
    plt.tight_layout()

__Plot true model__

In [ ]:
plt.close('all')
plt.figure()
do_plot(1, np.flipud(sigma0), 'seismic', '(a)', r"$\rm{\sigma [mS/m]}$", np.min(sigma0), np.max(sigma0))
do_plot(2, np.flipud(epsilonr0), 'seismic', '(b)', r"$\rm{\epsilon_r []}$", np.min(epsilonr0), np.max(epsilonr0))
#plt.savefig('test.png', format='png', dpi=100)
plt.savefig('2_cross_TE_model_true.pdf', bbox_inches='tight', format='pdf')
plt.show()

**Define acquisition geometry**

In [ ]:
# place receivers (top)
nrec = 30
xrec_1 = 2.0
xrec_2 = 11.0
drec = (xrec_2 - xrec_1) / nrec
xrec_top = np.arange(xrec_1, xrec_2, drec)
yrec_top = (xrec_top / xrec_top) * 2.0

# place receivers (bottom)
xrec_bot = np.arange(xrec_1, xrec_2, drec)
yrec_bot = (xrec_bot / xrec_bot) * 10.0

# place receivers (left)
yrec_1 = 2.0
yrec_2 = 10.0
drec = (yrec_2 - yrec_1) / (nrec-1)
yrec_left = np.arange(yrec_1 + drec, yrec_2, drec)
xrec_left = (yrec_left / yrec_left) * 2.0

# place receivers (right)
yrec_right = np.arange(yrec_1, yrec_2, drec)
xrec_right = (yrec_right / yrec_right) * 11.0

xrec = np.concatenate((xrec_top, xrec_bot, xrec_left, xrec_right), axis=0)
yrec = np.concatenate((yrec_top, yrec_bot, yrec_left, yrec_right), axis=0)

# assemble vectors into an array
tmp = np.zeros(xrec.size, dtype=[('var1', float), ('var2', float)])
tmp['var1'] = xrec
tmp['var2'] = yrec

# write receiver positions to file
np.savetxt('receiver_2_cross_TE.dat', tmp, fmt='%4.2f %4.2f')

In [ ]:
# place sources (top)
nrec = 10
xrec_1 = 2.0
xrec_2 = 11.0
drec = (xrec_2 - xrec_1) / nrec
xsrc_top = np.arange(xrec_1, xrec_2, drec)
ysrc_top = (xsrc_top / xsrc_top) * 2.0

# place sources (bottom)
xsrc_bot = np.arange(xrec_1, xrec_2, drec)
ysrc_bot = (xsrc_bot / xsrc_bot) * 10.0

# place sources (left)
yrec_1 = 2.0
yrec_2 = 10.0
drec = (yrec_2 - yrec_1) / nrec
ysrc_left = np.arange(yrec_1 + drec, yrec_2, drec)
xsrc_left = (ysrc_left / ysrc_left) * 2.0

# place sources (right)
ysrc_right = np.arange(yrec_1, yrec_2 + drec, drec)
xsrc_right = (ysrc_right / ysrc_right) * 11.0

xsrc = np.concatenate((xsrc_top, xsrc_bot, xsrc_left, xsrc_right), axis=0)
ysrc = np.concatenate((ysrc_top, ysrc_bot, ysrc_left, ysrc_right), axis=0)

nshot = 40

# assemble vectors into an array
tmp1 = np.zeros(nshot, dtype=[('var1', float), ('var2', float), ('var3', float), ('var4', float), 
                                  ('var5', float), ('var6', float), ('var7', float), ('var8', float)])
tmp1['var1'] = xsrc
tmp1['var2'] = 0.0 * (xsrc / xsrc)     # z-coordinate = 0 due to 2D code
tmp1['var3'] = ysrc
tmp1['var4'] = 0.0 * (xsrc / xsrc)     # time delay
tmp1['var5'] = 10.0 * (xsrc / xsrc)    # center frequency of source wavelet
tmp1['var6'] = 1.0 * (xsrc / xsrc)     # amplitude of source wavelet
tmp1['var7'] = 0.0 * (xsrc / xsrc)     # angle of rotated source
tmp1['var8'] = 3 * (xsrc / xsrc)       # source type = 3 (vertical impact source)


# write receiver positions to file
np.savetxt('source_2_cross_TE.dat', tmp1, fmt='%4.2f %4.2f %4.2f %4.2f %4.2f %4.2f %4.2f %4.2f')

__Plot model with acquisition geometry__

In [ ]:
plt.figure(figsize=(7, 6))

plt.imshow(np.flipud(epsilonr0), cmap='seismic', interpolation='none', extent=[DH,NX*DH,DH,NY*DH])
plt.hold(True)
plt.plot(xrec,yrec,'cv',markersize=5)
plt.plot(xsrc,ysrc,'r*',markersize=10)

a = gca()
a.set_xticklabels(a.get_xticks(), font)
a.set_yticklabels(a.get_yticks(), font)

plt.ylabel('Depth [m]', fontdict=font)
plt.xlabel('Distance [m]', fontdict=font)
plt.gca().invert_yaxis()
plt.axis('tight')
cbar=plt.colorbar(aspect=16, pad=0.01)
cbar.set_label(r"$\rm{\epsilon_r []}$", fontdict=font, labelpad=10)
plt.tight_layout()
#plt.savefig('test.png', format='png', dpi=100)
plt.savefig('2_cross_TE_acq.pdf', bbox_inches='tight', format='pdf')
plt.show()

__Compute true material parameters__

In [ ]:
# convert relative permittivity to permittivity
epsilon00 = 8.85e-12
epsilon0 = epsilonr0 * epsilon00

sigma0 = sigma0 * 1e-3

**Write models to file**

In [ ]:
# permittivity
name_taper = "2_cross_TE_true.eps"
f = open (name_taper, mode='wb')
data_type = np.dtype ('float32').newbyteorder ('<')
epsilon0 = np.array(epsilon0, dtype=data_type)
epsilon0 = np.rot90(epsilon0,1)
epsilon0 = np.flipud(epsilon0)
epsilon0.tofile(f)

# conductivity
name_taper = "2_cross_TE_true.sig"
f = open (name_taper, mode='wb')
data_type = np.dtype ('float32').newbyteorder ('<')
sigma0 = np.array(sigma0, dtype=data_type)
sigma0 = np.rot90(sigma0,1)
sigma0 = np.flipud(sigma0)
sigma0.tofile(f)